In [1]:
import pandas as pd, numpy as np

# FCC Internet Speed Data (Updated with 2016 data)

Internet providers report speeds to the FCC via form 477. 

* Data: https://www.fcc.gov/general/broadband-deployment-data-fcc-form-477
* Glossary: https://www.fcc.gov/general/explanation-broadband-deployment-data

# Part 0: Importing data

In [2]:
tracts = pd.read_csv("data/tracts_to_towns.csv")
tracts.head()

,tract,town
0,9001010101,Greenwich
1,9001010102,Greenwich
2,9001010201,Greenwich
3,9001010202,Greenwich
4,9001010300,Greenwich


In [3]:
providers = pd.read_excel("data/2016/Provider_List_Jun2016_v1.xlsx")
providers.head()

,FRN,ProviderName,StateAbbr,TechCode,records,blocks
0,10827,Veracity Networks,UT,50,4008,4008
1,12542,SERVICE ONE CABLE,LA,42,726,726
2,12781,"NEU VENTURES, INC.",TX,41,921,921
3,12781,"NEU VENTURES, INC.",TX,70,1093,1093
4,12849,"MUTUAL COMMUNICATIONS SERVICES, INC.",IA,41,21,21


In [4]:
ct_speeds = pd.read_csv("data/2016/CT-Fixed-Jun2016-v1.csv")
print len(ct_speeds["BlockCode"].unique())
ct_speeds.head()

65901


,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp
0,234293,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010907002008,50,0,0.0,0.0,1,60.0,60.0
1,235491,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010101014013,30,0,0.0,0.0,1,3.0,3.0
2,235492,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010101023018,30,0,0.0,0.0,1,1.5,1.5
3,235493,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010102011024,30,0,0.0,0.0,1,1.5,1.5
4,235494,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010102023002,30,0,0.0,0.0,1,1.5,1.5


# Part 0-1: Joining data

I'm going to add a town column to the ct_speeds dataframe, translating each tract to the town its a part of.

In [5]:
def list_of(x, n):
    return [x] * n

# Example usage 
print list_of(False, 5) + (list_of(False, 5))
print list_of(False, 10)


[False, False, False, False, False, False, False, False, False, False]
[False, False, False, False, False, False, False, False, False, False]


In [6]:
# Clean town names
tracts["town"] = tracts.apply(lambda x: x["town"].title().strip(), axis=1)
tracts.head()


,tract,town
0,9001010101,Greenwich
1,9001010102,Greenwich
2,9001010201,Greenwich
3,9001010202,Greenwich
4,9001010300,Greenwich


In [7]:

# Add a column of census tracts, shorter than the block codes
def code_to_tract(code):
    sample_tract = "9001010101"
    tract_length = len(sample_tract)
    # return int(str(code)[:tract_length])
    return int('{0:d}'.format(int(code))[:tract_length])

code_to_tract(90117051013000)

ct_speeds["tract"] = ct_speeds.apply(lambda x: code_to_tract(x["BlockCode"]), axis=1)
ct_speeds.head()

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp,tract
0,234293,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010907002008,50,0,0.0,0.0,1,60.0,60.0,9001090700
1,235491,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010101014013,30,0,0.0,0.0,1,3.0,3.0,9001010101
2,235492,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010101023018,30,0,0.0,0.0,1,1.5,1.5,9001010102
3,235493,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010102011024,30,0,0.0,0.0,1,1.5,1.5,9001010201
4,235494,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010102023002,30,0,0.0,0.0,1,1.5,1.5,9001010202


In [8]:
ct_speeds = ct_speeds.merge(tracts, left_on="tract", right_on="tract")
ct_speeds.head()


,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp,tract,town
0,234293,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010907002008,50,0,0.0,0.0,1,60.0,60.0,9001090700,Trumbull
1,2180866,21193,6275945,"XO Communications Services, LLC",XO Communications,"XO Holdings, Inc.",131513,"XO Holdings, Inc.",CT,90010907002008,30,0,0.0,0.0,1,20.0,20.0,9001090700,Trumbull
2,5778754,22328,18589226,"Lightower Fiber Networks I, LLC (fka Light Tow...",Lightower,LTS Group Holdings LLC,131095,LTS Group Holdings LLC,CT,90010907001000,50,0,0.0,0.0,1,1000.0,1000.0,9001090700,Trumbull
3,5778755,22328,18589226,"Lightower Fiber Networks I, LLC (fka Light Tow...",Lightower,LTS Group Holdings LLC,131095,LTS Group Holdings LLC,CT,90010907001001,50,0,0.0,0.0,1,1000.0,1000.0,9001090700,Trumbull
4,5778756,22328,18589226,"Lightower Fiber Networks I, LLC (fka Light Tow...",Lightower,LTS Group Holdings LLC,131095,LTS Group Holdings LLC,CT,90010907001002,50,0,0.0,0.0,1,1000.0,1000.0,9001090700,Trumbull


In [9]:
# checksum
print ct_speeds["town"].value_counts().sum()
print len(ct_speeds.index)

464115
464115


# Part 3: Analysis

In [10]:
def gen_report(town_df):
    print "There are " + str(len(town_df["BlockCode"].unique())) + " census blocks and "\
    + str(len(town_df["tract"].unique())) + " tracts in __."
    print str(len(town_df["DBAName"].unique())) + " companies provide service."
    
    print "Advertised max speeds range from "\
    + str(town_df["MaxAdUp"].min()) + "/" + str(town_df["MaxAdDown"].min())\
    + " to "\
    + str(town_df["MaxAdUp"].max()) + "/" + str(town_df["MaxAdDown"].max())
    
    return town_df

def town_report(town_name):
    print "Generating report for " + town_name
    return gen_report(ct_speeds[ct_speeds["town"] == town_name.title()])

town_report("Greenwich").head()

Generating report for Greenwich
There are 1242 census blocks and 15 tracts in __.
21 companies provide service.
Advertised max speeds range from 0.0/0.0 to 100.0/115.0


,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp,tract,town
626,235491,20901,4335584,MCI Communications Corporation,MCI,Verizon Communications Inc.,131425,Verizon Communications Inc.,CT,90010101014013,30,0,0.0,0.0,1,3.0,3.0,9001010101,Greenwich
627,3252080,21550,22757645,"dishNET Satellite Broadband, L.L.C.",dishNET Satellite Broadband LLC,"dishNET Holding, L.L.C.",130627,"dishNET Holding, LLC",CT,90010101013042,60,1,10.0,2.0,0,0.0,0.0,9001010101,Greenwich
628,4844489,22266,19027440,CSC Holdings LLC,Altice USA,"CSC Holdings, LLC",130370,Altice,CT,90010101011000,42,1,101.0,35.0,1,0.0,0.0,9001010101,Greenwich
629,4844490,22266,19027440,CSC Holdings LLC,Altice USA,"CSC Holdings, LLC",130370,Altice,CT,90010101011001,42,1,101.0,35.0,1,0.0,0.0,9001010101,Greenwich
630,4844491,22266,19027440,CSC Holdings LLC,Altice USA,"CSC Holdings, LLC",130370,Altice,CT,90010101011002,42,1,101.0,35.0,1,0.0,0.0,9001010101,Greenwich


In [11]:
def summary_table(df, col):
    return ct_speeds.groupby(col)\
    .agg({
            "MaxAdDown":max,
            "MaxAdUp":max,
            "MaxCIRDown":max,
            "MaxCIRUp":max,
            "DBAName":lambda x: x.nunique(),
#             "BlockCode":lambda x: x.nunique(),   
         })\
    .reset_index()

town_speeds = summary_table(ct_speeds, "town")

town_speeds.head()

,town,DBAName,MaxAdDown,MaxAdUp,MaxCIRDown,MaxCIRUp
0,Ansonia,12,150.0,35.0,1000.0,1000.0
1,Ashford,8,115.0,12.0,1000.0,1000.0
2,Avon,14,150.0,12.0,1000.0,1000.0
3,Barkhamsted,9,115.0,12.0,1000.0,1000.0
4,Beacon Falls,8,150.0,12.0,1000.0,1000.0


#### Top business speeds: you can get 1 Gb everywhere

Every town offers 1000 Mbps up/down for businesses.

MaxCIRDown/Up - Maximum contractual downstream[/upstream] bandwidth offered by the provider in the block for Business service (filer directed to report 0 if the contracted service is sold on a "best efforts" basis without a guaranteed data-throughput rate)



In [12]:
town_speeds["MaxCIRUp"].value_counts()

1000.0    167
Name: MaxCIRUp, dtype: int64

In [13]:
town_speeds["MaxCIRDown"].value_counts()

1000.0    167
Name: MaxCIRDown, dtype: int64

#### Max advertised speed per town

In [14]:
town_speeds[["town","MaxAdUp","MaxAdDown"]].sort_values(by="MaxAdDown")

,town,MaxAdUp,MaxAdDown
122,Scotland,5.0,100.0
17,Brooklyn,5.0,100.0
21,Chaplin,5.0,100.0
43,Eastford,5.0,100.0
60,Hampton,5.0,100.0
65,Kent,35.0,101.0
52,Goshen,35.0,101.0
146,Warren,35.0,101.0
147,Washington,35.0,101.0
84,Morris,35.0,101.0


In [15]:
summary_table(ct_speeds[(ct_speeds["Consumer"] == 1)\
                       &(ct_speeds["MaxAdDown"] > 0)],
              "BlockCode")["MaxAdDown"].value_counts()

150.0     22156
115.0      8716
101.0      7944
100.0      6912
15.0       6369
300.0      5529
24.0       4969
120.0      2603
110.0       526
0.0         103
12.0         33
6.0          20
1.0          10
55.0          5
1000.0        5
10.0          1
Name: MaxAdDown, dtype: int64

In [16]:
town_speeds.sort_values(by="MaxAdDown")

,town,DBAName,MaxAdDown,MaxAdUp,MaxCIRDown,MaxCIRUp
122,Scotland,7,100.0,5.0,1000.0,1000.0
17,Brooklyn,10,100.0,5.0,1000.0,1000.0
21,Chaplin,9,100.0,5.0,1000.0,1000.0
43,Eastford,9,100.0,5.0,1000.0,1000.0
60,Hampton,8,100.0,5.0,1000.0,1000.0
65,Kent,11,101.0,35.0,1000.0,1000.0
52,Goshen,9,101.0,35.0,1000.0,1000.0
146,Warren,9,101.0,35.0,1000.0,1000.0
147,Washington,12,101.0,35.0,1000.0,1000.0
84,Morris,8,101.0,35.0,1000.0,1000.0


#### How much "competition?"

There were an average of 12 providers, with as many as 24 and as few as 7 providers in each CT town.

In [17]:
town_speeds["DBAName"].describe()

count    167.000000
mean      12.766467
std        3.646993
min        7.000000
25%       10.000000
50%       12.000000
75%       16.000000
max       22.000000
Name: DBAName, dtype: float64

In [18]:
summary_table(ct_speeds, "tract")["DBAName"].describe()

count    829.000000
mean      10.118215
std        2.268354
min        6.000000
25%        8.000000
50%       10.000000
75%       11.000000
max       19.000000
Name: DBAName, dtype: float64

#### Businesses have a lot of choices
That's a surprisingly high number of providers, when I know that in practice it's nearly impossible as a consumer to get more than one company to provide internet access to an address, let alone 12. What if we exclude business providers?

In [19]:
summary_table(ct_speeds[ct_speeds["Consumer"] == 1], "BlockCode")["DBAName"].describe()

count    65901.000000
mean         5.967451
std          1.220557
min          1.000000
25%          6.000000
50%          6.000000
75%          7.000000
max         15.000000
Name: DBAName, dtype: float64

#### Still higher than expected...

I still didn't expect to see an average of 6 companies providing consumer internet acess in a given census block.

Despite what the data has checked off, I don't believe these are all truly consumer ISPs. https://www.megapath.com/ describes itself as a business internet provider.

In [20]:
def list_providers(town_name):
    return ct_speeds[(ct_speeds["town"] == town_name.title())
                    & (ct_speeds["Consumer"] == 1)]["DBAName"].unique()

list_providers("Bethel")



array(['dishNET Satellite Broadband LLC', 'Comcast',
       'Frontier Communications Corporation', 'Global Capacity LLC',
       'ViaSat Inc', 'HughesNet', 'Skycasters',
       'Charter Communications Inc', 'Altice USA'], dtype=object)

In [21]:
providers[providers["StateAbbr"] == "CT"]\
.drop_duplicates(subset=["ProviderName"]).sort_values(by="blocks", ascending=False)

,FRN,ProviderName,StateAbbr,TechCode,records,blocks
76,1568880,GCI Communication Corp.,CT,60,64389,64389
5879,12369286,"HNS License Sub, LLC",CT,60,64389,64389
4263,4963088,"ViaSat, Inc.",CT,60,64389,64389
7409,18756155,"VSAT Systems, LLC",CT,60,64389,64389
1258,3576352,Frontier Communications Corporation,CT,10,49955,49955
2697,3768165,"COMCAST CABLE COMMUNICATIONS, LLC",CT,42,22940,22940
7131,18589226,"Lightower Fiber Networks I, LLC (fka Light Tow...",CT,50,20968,20968
7528,19027440,CSC Holdings LLC,CT,42,14217,14217
2288,3746468,"Charter Communications, Inc.",CT,42,9004,9004
564,1834696,"Cox Communications, Inc",CT,42,5724,5724


In [22]:
ct_consumer_speeds = ct_speeds[ct_speeds["Consumer"] == 1]

def dbas_in(provider_name, col="tract", df=ct_consumer_speeds):
    return ct_consumer_speeds[(df["DBAName"] == provider_name)]\
.drop_duplicates(subset=[col])

def count_in(provider_name,col="tract"):
    return len(dbas_in('Frontier Communications Corporation',
                       col=col)[col].index)


print count_in('Frontier Communications Corporation',"BlockCode")
print count_in('Frontier Communications Corporation',"tract")
print count_in('Frontier Communications Corporation',"town")

53193
820
167


In [23]:
def test():
    ct_consumer_speeds = ct_speeds[ct_speeds["Consumer"] == 1]
    ct_non_consumer_speeds = ct_speeds[ct_speeds["Consumer"] == 0]
    
    print "Consumer records:" + str(len(ct_consumer_speeds.index))
    print "Non-consumer records:" + str(len(ct_non_consumer_speeds.index))
test()

Consumer records:373037
Non-consumer records:91078


In [24]:
def blocks_in_state(dba):
    return ct_consumer_speeds[(ct_consumer_speeds["DBAName"] == dba)]

def blocks_in_town(dba):
    df = blocks_in_state(dba)
    return df[(df["town"] == 'Bethel')]

def tracts_in_state(dba):
    return blocks_in_state(dba).drop_duplicates(subset=["tract"])

def towns_in_state(dba):
    return blocks_in_state(dba).drop_duplicates(subset=["town"])

def max_upload(df):
    return df["MaxAdUp"].max()

def max_download(df):
    return df["MaxAdDown"].max()

def row_count(df):
    return len(df.index)

row_count(blocks_in_state('Charter Communications Inc'))

9004

In [25]:
def all_providers ():
    provider_names = []
    provider_count_in_towns = []
    provider_count_in_blocks = []
    ret = []
    for dba in ct_consumer_speeds\
    .drop_duplicates(subset="DBAName")["DBAName"]:
        ret.append([
                dba,
                row_count(towns_in_state(dba)),
                row_count(tracts_in_state(dba)),
                row_count(blocks_in_state(dba)),
                max_upload(blocks_in_state(dba)),
                max_download(blocks_in_state(dba))
            ])
        
#         provider_names.append(dba)
#         provider_count_in_towns.append(count_in(dba, "town"))
#         provider_count_in_blocks.append(count_in(dba,"BlockCode"))
#     return [provider_names,provider_count_in_towns,provider_count_in_blocks]
    return pd.DataFrame(ret,columns=["Provider",
                                     "Consumer Towns",
                                     "Consumer Tracts",
                                     "Consumer Blocks",
                                    "Max Upstream",
                                    "Max Downstream"])

all_providers().sort_values(by="Consumer Blocks",
                           ascending=False)


,Provider,Consumer Towns,Consumer Tracts,Consumer Blocks,Max Upstream,Max Downstream
2,Frontier Communications Corporation,167,820,122906,30.0,115.0
3,ViaSat Inc,167,829,64389,3.0,12.0
4,HughesNet,167,829,64389,2.0,15.0
5,Skycasters,167,829,64389,1.3,2.0
0,Comcast,107,465,22197,10.0,150.0
7,Altice USA,49,236,14217,35.0,101.0
1,Charter Communications Inc,68,125,9004,5.0,100.0
11,Cox Communications,28,117,5538,1000.0,1000.0
13,"Atlantic Broadband Finance, LLC",17,39,2612,10.0,120.0
14,Thames Valley Communications,3,16,1529,20.0,110.0


In [26]:
ct_speeds[ct_speeds["DBAName"]=="Frontier Communications Corporation"]["MaxAdUp"].max()
ct_speeds[ct_speeds["DBAName"]=="Frontier Communications Corporation"]["MaxAdDown"].max()

115.0

#### Differences by blocks and tracks

In [27]:
blocks = ct_speeds["BlockCode"].drop_duplicates().sort_values().to_frame()
tracts_data = ct_speeds["tract"].drop_duplicates().sort_values().to_frame()

In [28]:
def tract_df (tract):
    return ct_speeds[ct_speeds["tract"] == tract]
def block_df(block):
    return ct_speeds[ct_speeds["BlockCode"] == block]

def max_download_in_block(block):
    return max_download(block_df(block))

def max_download_in_tract(tract):
    return max_download(tract_df(tract))

In [31]:
tracts_data["MaxAdDown"] = tracts_data.apply(lambda x: max_download_in_tract(x["tract"]),axis=1)
tracts_data["MaxAdDown"].value_counts()

150.0     438
115.0     203
300.0     115
120.0      34
101.0      22
100.0      14
1000.0      2
24.0        1
Name: MaxAdDown, dtype: int64

In [32]:
blocks["MaxAdDown"] = blocks.apply(lambda x: max_download_in_block(x["BlockCode"]),axis=1)
blocks["MaxAdUp"] = blocks.apply(lambda x: max_download_in_block(x["BlockCode"]),axis=1)
blocks["MaxAdDown"].value_counts()

150.0     22156
115.0      8716
101.0      7944
100.0      6912
15.0       6369
300.0      5529
24.0       4969
120.0      2603
110.0       526
0.0         103
12.0         33
6.0          20
1.0          10
55.0          5
1000.0        5
10.0          1
Name: MaxAdDown, dtype: int64

In [33]:
row_count(blocks[blocks["MaxAdDown"] < 25])

11505

In [34]:
row_count(blocks[blocks["MaxAdDown"] >= 25])

54396

In [35]:
blocks.columns

Index([u'BlockCode', u'MaxAdDown', u'MaxAdUp'], dtype='object')

In [36]:
#blocks.apply(lambda x: "{0:d}".format(int(x["BlockCode"])),axis=1).to_frame()
blocks["tract"] = blocks.apply(lambda x: code_to_tract(x["BlockCode"]),axis=1).to_frame()
blocks.head()

,BlockCode,MaxAdDown,MaxAdUp,tract
628,90010101011000,101.0,101.0,9001010101
629,90010101011001,101.0,101.0,9001010101
630,90010101011002,101.0,101.0,9001010101
631,90010101011003,101.0,101.0,9001010101
632,90010101011004,101.0,101.0,9001010101


In [37]:
#blocks[blocks["MaxAdDown"] < 25].to_csv("output/slow_blocks.csv")

In [38]:
#tracts["town"] = tracts.apply(lambda x: x["town"].title().strip(), axis=1)


In [39]:
blocks = blocks.merge(tracts,left_on="tract",right_on="tract")
blocks.head()

,BlockCode,MaxAdDown,MaxAdUp,tract,town
0,90010101011000,101.0,101.0,9001010101,Greenwich
1,90010101011001,101.0,101.0,9001010101,Greenwich
2,90010101011002,101.0,101.0,9001010101,Greenwich
3,90010101011003,101.0,101.0,9001010101,Greenwich
4,90010101011004,101.0,101.0,9001010101,Greenwich


In [40]:
# Identify each town's percentage of slow blocks
# A slow block fails to meet the definition of broadband of 25 Mbps down / 3 up
def pct_slow(df):
    total_blocks = len((df["BlockCode"].unique()))
    #print "total blocks:", total_blocks
    tmpdf = df[(df["MaxAdUp"] > 0) | (df["MaxAdDown"] > 0)]
    tmpdf = tmpdf[(df["MaxAdUp"] < 3) | (df["MaxAdDown"] < 25)]
    slow_blocks = len(tmpdf.index)
    #print "slow_blocks",slow_blocks
    try:
        return [slow_blocks, total_blocks, slow_blocks * 100 / total_blocks]
    except:
        return [0,0,0]

def pct_slow_town(town_name):
    return pct_slow(blocks[blocks["town"] == town_name])

def slow_towns():
    ret = []
    for t in tracts["town"].unique():
        ret.append([t] + pct_slow_town(t))
    return pd.DataFrame(ret, columns=["town","slow blocks","total blocks","percent slow"])

print slow_towns()[["town","percent slow","slow blocks","total blocks"]]\
.sort_values(by="percent slow", ascending=False)\
.to_csv(sep="\t",index=False)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


town	percent slow	slow blocks	total blocks
North Stonington	52	81	153
Salisbury	47	140	292
Hartland	47	45	94
Chester	45	72	159
East Granby	44	61	138
Madison	44	213	478
Sharon	44	114	255
Norfolk	42	128	298
Lyme	41	67	162
Haddam	41	136	328
Portland	40	95	234
Preston	39	73	187
Suffield	39	35	88
Windsor Locks	39	111	283
Windsor	38	188	489
Salem	38	47	121
Essex	38	65	171
Stafford	38	181	465
Cromwell	37	99	265
Bloomfield	37	194	522
East Hampton	36	99	274
Bozrah	36	46	126
Franklin	35	26	73
Middlefield	34	80	229
Colchester	34	115	329
Middlebury	33	89	266
Marlborough	33	53	157
Rocky Hill	32	102	318
Branford	32	171	522
Beacon Falls	32	32	99
Berlin	32	135	421
Middletown	32	216	657
New London	32	162	500
East Haddam	31	109	342
Farmington	31	136	435
Glastonbury	30	185	601
Avon	30	106	349
Clinton	30	79	259
Deep River	30	38	123
North Canaan	29	35	119
North Haven	29	147	505
Durham	29	46	157
Sterling	29	34	116
Lisbon	28	30	106
Simsbury	28	139	492
Killingly	28	156	545
Hartford	28	329	1150
Old Lyme	28	41	

In [41]:
slow_pct_towns = slow_towns()[["town","percent slow","slow blocks","total blocks"]]
slow_pct_towns.sort_values("percent slow", ascending=False)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,town,percent slow,slow blocks,total blocks
132,North Stonington,52,81,153
57,Salisbury,47,140,292
23,Hartland,47,45,94
87,Chester,45,72,159
35,East Granby,44,61,138
111,Madison,44,213,478
58,Sharon,44,114,255
78,Norfolk,42,128,298
121,Lyme,41,67,162
85,Haddam,41,136,328


In [42]:
print slow_pct_towns["total blocks"].sum()
print slow_pct_towns["slow blocks"].sum()
print slow_pct_towns["slow blocks"].sum() * 100 / slow_pct_towns["total blocks"].sum()

65901
11402
17
